In [1]:
import netCDF4
import pandas as pd
from netCDF4 import num2date
import numpy as np
from datetime import datetime, timedelta
import cftime

In [2]:
# 文件路径
nc_file_path = 'E:\\DownLoad\\TotalP\\02.nc'
excel_file_path = 'E:\\DownLoad\\TotalP\\tp.xlsx'

# 经度和纬度范围
lon_min, lon_max = 45, 46  # 经度范围
lat_min, lat_max = 59, 60    # 纬度范围

# UTC+8 时间范围
start_time_utc8 = '2023-01-01 12:00:00'  # 开始时间 (UTC+8)
end_time_utc8 = '2023-01-03 20:00:00'    # 结束时间 (UTC+8)

In [3]:
# 打开nc文件
dataset = netCDF4.Dataset(nc_file_path)

# 获取时间、经度和纬度变量
time_var = dataset.variables['time']
time = time_var[:]
longitude = dataset.variables['longitude'][:]
latitude = dataset.variables['latitude'][:]
tp = dataset.variables['tp'][:]  # (time, latitude, longitude)

In [4]:
# 获取时间单位
time_units = time_var.units

# 将时间变量转换为 UTC 时间
time_dates_utc = num2date(time, units=time_units, calendar='gregorian')

# 定义时间偏移量
time_offset = timedelta(hours=8)  # UTC+8

# 将 UTC+8 时间范围转换为 UTC 时间范围
start_time_utc8_dt = datetime.strptime(start_time_utc8, '%Y-%m-%d %H:%M:%S')
end_time_utc8_dt = datetime.strptime(end_time_utc8, '%Y-%m-%d %H:%M:%S')

start_time_utc_dt = start_time_utc8_dt - time_offset
end_time_utc_dt = end_time_utc8_dt - time_offset

# 将转换后的时间范围转换为与 netCDF4 时间变量兼容的时间对象
start_time_utc = cftime.date2num(start_time_utc_dt, units=time_units, calendar='gregorian')
end_time_utc = cftime.date2num(end_time_utc_dt, units=time_units, calendar='gregorian')

In [5]:
# 找到时间范围内的索引
time_indices = np.where((time >= start_time_utc) & (time <= end_time_utc))[0]

# 筛选时间范围内的数据
tp_filtered = tp[time_indices, :, :]
time_dates_filtered = time_dates_utc[time_indices]

# 找到经纬度范围内的索引
lon_indices = np.where((longitude >= lon_min) & (longitude <= lon_max))[0]
lat_indices = np.where((latitude >= lat_min) & (latitude <= lat_max))[0]

# 筛选经纬度范围内的数据
tp_filtered = tp_filtered[:, lat_indices, :][:, :, lon_indices]
filtered_latitude = latitude[lat_indices]
filtered_longitude = longitude[lon_indices]

In [6]:
# 创建一个ExcelWriter对象
with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
    for t_index, utc_date in enumerate(time_dates_filtered):
        # 转换为 UTC+8 时间
        local_date = utc_date + time_offset

        # 获取当前时间步的数据
        tp_data = tp_filtered[t_index, :, :]

        # 创建DataFrame
        df = pd.DataFrame(tp_data, index=filtered_latitude, columns=filtered_longitude)
        
        # 创建工作表名称
        sheet_name = local_date.strftime('%Y-%m-%d_%H-%M-%S')  # 格式化为字符串
        
        # 将DataFrame写入不同的工作表
        df.to_excel(writer, sheet_name=sheet_name)

print(f'数据已成功保存到 {excel_file_path}')

数据已成功保存到 E:\DownLoad\TotalP\tp.xlsx
